# 3. Converting data to OME-NGFF (practical)

***

**ELMI 2021 NGFF Workshop**, 25 June 2021

***


## Summary

* 3.1. Data from IDR
* 3.2. Converting your data to OME-NGFF
* 3.3. Publishing your data with S3
* 3.4. Extras (time-permitting)

***

## 3.1. Data from S3
We're going to start off by looking at some images you will likely have seen during the OMERO or IDR sessions.

**Our goal is to share these *without* using an OMERO.**

<table>
    <tr>
        <td>
            <img alt="idr0062 thumbnails" src="images/training-1.png" style="height:150px"/>
        </td>
        <td>
            <img alt="idr0062 thumbnails" src="images/training-2.png" style="height:150px"/>
        </td>
        <td>
            <img alt="idr0023 3D screenshot" src="images/training-3.png" style="height:150px"/>
        </td>
    </tr>
</table>
    
The left two images are from  the ilastik plugin guide presented by Petr: https://omero-guides.readthedocs.io/en/latest/ilastik/docs/ilastik_fiji.html

They are available in the "idr0062" project on the workshop server: https://workshop.openmicroscopy.org/webclient/?show=project-1952

The original dataset can be found in IDR study idr0062 by Blin _et al._: https://idr.openmicroscopy.org/webclient/?show=project-801

The image on the right is from idr0023 by Szymborska _et al_: http://idr.openmicroscopy.org/webclient/?show=project-52 and is **much** smaller.


***

## 3.2 Converting your data to OME-NGFF

The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.


### 3.2.1 Conversion tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4


<img src="images/conversion.png" style="height:400px" />

In [1]:
!bioformats2raw --version

Version = 0.3.0-rc4
Bio-Formats version = 6.5.1


In [2]:
!bioformats2raw

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [-p] [--no-hcs] [--[no-]nested] [--no-root-group]
                    [--overwrite] [--version] [--debug[=<logLevel>]]
                    [--extra-readers[=<extraReaders>[,<extraReaders>...]]]...
                    [--options[=<readerOptions>[,<readerOptions>...]]]... [-s
                    [=<seriesList>[,<seriesList>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--dimension-order=<dimensionOrder>]
                    [--downsample-type=<downsampling>]
                    [--fill-value=<fillValue>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>]
                    [--memo-directory=<memoDirectory>]
                    [--pixel-type=<outputPixelType>]
                    [--pyramid-name=<pyramidName>] [-r=<pyr

In [8]:
!java --version

openjdk 11.0.9.1 2020-11-04 LTS
OpenJDK Runtime Environment Zulu11.43+55-CA (build 11.0.9.1+1-LTS)
OpenJDK 64-Bit Server VM Zulu11.43+55-CA (build 11.0.9.1+1-LTS, mixed mode)


In [35]:
import os
os.environ["JAVA_OPTS"]="--illegal-access=deny"

In [37]:
%%time
!bioformats2raw --overwrite trans_norm.tif trans_norm.ome.zarr

2021-06-23 14:31:54,862 [main] WARN  c.g.bioformats2raw.Converter - Overwriting output path trans_norm.ome.zarr
CPU times: user 42.9 ms, sys: 18.3 ms, total: 61.2 ms
Wall time: 3.19 s


In [38]:
!find trans_norm.ome.zarr -name ".z*"

trans_norm.ome.zarr/.zattrs
trans_norm.ome.zarr/.zgroup
trans_norm.ome.zarr/0/.zattrs
trans_norm.ome.zarr/0/.zgroup
trans_norm.ome.zarr/0/0/.zarray


In [40]:
!ls -ltrah trans_norm.ome.zarr/0/0/0/0/0/0/0

-rw-r--r--  1 jamoore  wheel   1.2K Jun 23 14:31 trans_norm.ome.zarr/0/0/0/0/0/0/0


In [41]:
!ome_zarr info trans_norm.ome.zarr/0

/opt/NGFF-ELMI-2021-Workshop/trans_norm.ome.zarr/0 [zgroup]
 - metadata
   - Multiscales
 - data
   - (1, 1, 571, 30, 30)


***

## 3.3. Publishing your data with S3

You can then move the generated output to S3. Note: one of the most frequent mistakes here is the slash (`/`) at the end of the commands.x

In [42]:
YOURNAME = input()

josh


In [43]:
!time mc cp --recursive trans_norm.ome.zarr/0/ elmi2021/idr-upload/elmi2021/$YOURNAME/my_trans_norm.ome.zarr/

.../0/99/0/0:  773.27 KiB / 773.27 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  23.94 KiB/s 32s
real	0m34.290s
user	0m0.697s
sys	0m0.577s


In [45]:
!mc cat elmi2021/idr-upload/elmi2021/$YOURNAME/my_trans_norm.ome.zarr/.zattrs

{
  "multiscales" : [
    {
      "metadata" : {
        "method" : "loci.common.image.SimpleImageScaler",
        "version" : "Bio-Formats 6.5.1"
      },
      "datasets" : [
        {
          "path" : "0"
        }
      ],
      "version" : "0.2"
    }
  ]
}

In [62]:
import getpass
import os

os.environ["FSSPEC_CONFIG_DIR"] = "/tmp"
os.environ["AWS_ACCESS_KEY_ID"] = "elmi2021"
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass()
with open("/tmp/conf.json", "w") as o:
    o.write("""
    {"s3":
        {"client_kwargs":
          {"endpoint_url": "https://idr-ftp.openmicroscopy.org"}
        }
    }""")

!ome_zarr -qqq info s3://idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/

········
DEBUG:s3fs:Setting up s3fs instance
DEBUG:s3fs:CALL: head_object - ({},) - {'Bucket': 'idr-upload', 'Key': 'elmi2021/josh/my_trans_norm.ome.zarr'}
DEBUG:s3fs:Get directory listing page for idr-upload/elmi2021/josh
DEBUG:s3fs:CALL: head_object - ({},) - {'Bucket': 'idr-upload', 'Key': 'elmi2021/josh/my_trans_norm.ome.zarr/.zarray'}
DEBUG:s3fs:Get directory listing page for idr-upload/elmi2021/josh/my_trans_norm.ome.zarr
DEBUG:s3fs:Get directory listing page for idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/.zarray
DEBUG:s3fs:Fetch: idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/.zgroup, 0-23
DEBUG:s3fs:Fetch: idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/.zattrs, 0-264
DEBUG:s3fs:CALL: head_object - ({},) - {'Bucket': 'idr-upload', 'Key': 'elmi2021/josh/my_trans_norm.ome.zarr/0/.zarray'}
DEBUG:s3fs:Fetch: idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/0/.zarray, 0-365
s3://idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/ [zgroup]
 - metadata
   - Multiscales
 - data
   - 

In [66]:
from IPython.display import Video
Video("images/idr0023.mp4")

# 3.4 Extras

## 3.4.1 Renaming

Another important distinction to filesystems is that though it looks like hello is in a directory, you should really think of the entire string after the bucket just as a "key".

In [67]:
!mc mv --recursive elmi2021/idr-upload/elmi2021/$YOURNAME/my_trans_norm.ome.zarr/ elmi2021/idr-upload/elmi2021/$YOURNAME/public_trans_norm.ome.zarr

mc: <ERROR> Unable to get bucket lock configuration of `elmi2021/idr-upload/elmi2021/josh/my_trans_norm.ome.zarr/`. Access Denied.


## 3.4.2 omero-cli-zarr

Loading the image from: https://workshop.openmicroscopy.org/webclient/img_detail/55500/?dataset=6210

In [25]:
# Another block to get your workshop password from day 1
import getpass
workshop_pass = getpass.getpass()

········


In [26]:
!omero login trainer-1@wss://workshop.openmicroscopy.org/omero-ws -w $workshop_pass

Previous session expired for trainer-1 on wss://workshop.openmicroscopy.org/omero-ws:4064
Created session for trainer-1@wss://workshop.openmicroscopy.org/omero-ws:443. Idle timeout: 10 min. Current group: Lab1


In [27]:
!time omero zarr export Image:55500

Using session for trainer-1@wss://workshop.openmicroscopy.org/omero-ws:443. Idle timeout: 10 min. Current group: Lab1
Exporting to 55500.zarr
Finished.

real	2m12.037s
user	0m5.787s
sys	0m1.844s


In [28]:
!find 55500.zarr -name ".z*"

55500.zarr/.zattrs
55500.zarr/.zgroup
55500.zarr/0/.zarray
55500.zarr/1/.zarray
55500.zarr/2/.zarray


## 3.4.3 Other resources

<table>
    <tr>
        <td>
            <a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">
<img src="images/resources-1.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">Diagram for how data moves</a>
        </td>
    </tr>
    <tr>
        <td>
      <a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">      
<img src="images/resources-2.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">Blog post for an easy way to publish OME-Zarr files</a>
        </td>
    </tr>
</table>    

### License (BSD 2-Clause)

Copyright (c) 2021, University of Dundee All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.